In [21]:
import os
import threading
import jieba
import sys

In [22]:
#新建一个MyThread类，继承threading.Thread   
class MyThread(threading.Thread):
    def __init__(self, filename, lock, progress):
        super().__init__()
        self.filename = filename
        self.lock = lock
        self.progress = progress

    def run(self):
        with open(self.filename, 'r', encoding='utf-8') as file:
            text = file.read()
            seg_list = jieba.cut(text)#使用jieba进行分词
            #利用过滤器函数删除一些无关的词语
            noun_set = set(filter(lambda x: x not in {"红楼梦", "分章回"}, seg_list))
            output_str = f"{self.filename}\n{'，'.join(noun_set)}\n"
            with open("cut_result.txt", 'a', encoding='utf-8') as output_file:
                output_file.write(output_str)

            #写入文件时需要加锁进行操作，线程同步
            with self.lock:
                self.progress[0] += 1
                update_progress(self.progress[0], self.progress[1])    
    

In [23]:
#监控进度，整体进度用进度条显示
def update_progress(completed_files, total_files):
    percentage = completed_files / total_files * 100
    sys.stdout.write('\r[{}] {:.2f}%'.format('#' * int(percentage / 2), percentage))
    sys.stdout.flush()


In [24]:
def main():
    folder_path = "红楼梦(分章回)"
    file_list = os.listdir(folder_path)
    total_files = len(file_list)
    
    progress = [0, total_files]  #使用列表保存进度信息
    lock = threading.Lock()
    threads = []

    for filename in file_list:
        file_path = os.path.join(folder_path, filename)
        thread = MyThread(file_path, lock, progress)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    #显示最终的进度信息
    update_progress(progress[1], progress[1])  
    sys.stdout.write('\n')
    sys.stdout.flush()

In [25]:
if __name__ == "__main__":
    main()

[##################################################] 100.00%
